# MEU_4_1


# 身份
你是一个精通法律的程序员. 擅长编写代码以检验特定案例对于特定法律要求的合规性. 

## MEU概念简述
MEU (Minimum Executable Unit) 是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东") 
- condition: 触发条件（如"减持股份")  
- constraint: 约束内容（如"提前15日公告") 
- contextual_info: 补充说明（如价格计算方式）


# 任务
你现在面临一个编写代码以检验某法律事件的合规性的任务, 你需要编写一个函数, 这个函数的输入是一个日频的pandas dataframe, 每一行是在某个交易日内公司的情况和股东的情况, 这个函数会检验该dataframe的每一行在某一个给定的法律最小可执行单元上的合规性, 标注在特定的列上, 并返回标注后的dataframe. 


# 更多要求
 - 你可以进行任意长度的思考, 然后用<CODE></CODE>包裹你的代码. 
 - 你需要按照下面这个框架编写代码: 1. 标记valid的subject(meu_n_k_subject); 2. 标记valid的condition(meu_n_k_condition); 3. 标记valid的constrain(meu_n_k_constrain); 4. 在dataframe中记录subject, condition和constrain的valid情况; 5. 计算整体的违约情况并标记在dataframe上(meu_n_k_compliance). 

# example
下面是一个例子:

## 输入
法律的最小可执行单元编号: MEU_4_1
法律的最小可执行单元内容: {"subject": "上市公司大股东 | 董监高", "condition": "计划通过本所集中竞价或大宗交易减持股份", "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", "contextual_info": NaN}
必要的额外信息: NaN

## 输出
<CODE>
import pandas a pd

def check_meu_4_1(df):
    '''
    代码的注释要清晰, 展示你分别验证subject, condition和constrain的过程.
    "subject": "上市公司大股东 | 董监高", 
    "condition": "计划通过本所集中竞价或大宗交易减持股份", 
    "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", 
    "contextual_info": NaN
    '''

    df['meu_4_2_subject'] = False  # 初始化subject列
    df['meu_4_2_condition'] = False  # 初始化condition列
    df['meu_4_2_constrain'] = False  # 初始化constrain列

    # 标记valid的subject
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'].isin(['董监高'])
    valid_subject = is_major_shareholder | is_director

    # 标记valid的condition
    valid_condition = df['减持方式'].isin(['竞价交易', '大宗交易'])

    # 标记valid的constrain
    df['交易日差'] = (df['计划开始日'] - df['计划披露日']).dt.days
    has_plan = df['存在减持计划']
    valid_constraint = df['交易日差'] >= 15 & df['存在减持计划']

    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_4_1_subject'] = True
    df.loc[valid_condition, 'meu_4_1_condition'] = True
    df.loc[valid_constraint, 'meu_4_1_constraint'] = True

    # 判断违规情况：满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )

    # 标记违规行
    df.loc[is_violation, 'meu_4_1_compliance'] = False

    return df

</CODE>

# 下面轮到你来完成这个任务
法律的最小可执行单元编号: MEU_12_3
法律的最小可执行单元内容: 上市公司控股股东 | 实际控制人 | 一致行动人 | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 | 不得存在下列情形：上市公司最近一期经审计的财务报告的归属于上市公司股东的净利润为负 | 
必要的额外信息: 无


# 附加信息
## 你可以操作的dataframe的columns: 
['日期', '日收益率', '收盘价', '前收盘价', '上市日期', '发行价格', '净资产增长率', '净利润增长率', '净资产',
'净利润', '总股本', '每股净资产', '每股净利润', '公告类型', '公告日期', '复权因子', '收盘价减每股净资产',
'收盘价减每股净利润', '公司涉嫌证券期货违法犯罪事件', '持股比例', '持股数量', '股东身份', '减持方式', '拟减持原因',
'存在减持计划', '计划减持比例', '计划价格下限', '计划价格上限', '当日减持比例', '累计减持比例', '计划披露日',
'计划开始日', '计划结束日', '离任日期', '公司简称', '股东', '股东涉嫌证券期货违法犯罪事件']

## 其中: 
- 所有日期相关的列都已经是pd.datetime格式, ['日期']列本身就是交易日, 不需要额外计算交易日, 但是自然日需要计算. 
- 所有价格的单位都是元而非万元, 千元或百万元; 所有的比例都是小数而非百分数
- 这是一个二维的面板数据, 根据['日期', '公司简称', '股东']可以定位到特定公司的特定股东在特定交易日的状态, 以及公司在该日的状态. 如果你要分公司和股东地考虑一些合规要求, 需要考虑按照 ['公司简称', '股东']分组并在组内完成. 
- 交易方式有['竞价交易', '大宗交易', '融券卖出'], 竞价交易包括集中竞价. 
- ['公司/股东涉嫌证券期货违法犯罪事件'] 有['被中国证监会及其派出机构立案调查', '中国证监会及其派出机构立案调查结束', '行政处罚决定作出', '被司法机关立案侦查', '司法机关立案侦查结束', '被本所公开谴责']
- ['股东身份'] 有 ['董监高', '离任董监高', '控股股东', '实际控制人', '持股5%以上股东', '普通股东'], 这些标签不会并列出现, 因此有时候董监高/离任董监高持股在5%以上但并没有标明'持股5%以上股东', 你需要从['持股比例']手动筛选. 
- ['存在减持计划']是一个布尔值，表示该股东当天是否有减持计划
- 假设公司上市发行价就是最早的非空的收盘价, 公司发行时的控股股东和实际控制人就是第一个非空的收盘价的同一天的控股股东和实际控制人. 
- ['持股比例'] 全部是公开上市前取得的股份. 不考虑承诺和限制出售的股份. 



In [ ]:
import pandas as pd

def check_meu_12_3(df):
    '''
    检查MEU_12_3合规性:
    "subject": "上市公司控股股东 | 实际控制人 | 一致行动人",
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的",
    "constraint": "不得存在下列情形：上市公司最近一期经审计的财务报告的归属于上市公司股东的净利润为负",
    "contextual_info": NaN
    '''
    
    # 初始化标记列
    df['meu_12_3_subject'] = False
    df['meu_12_3_condition'] = False
    df['meu_12_3_constraint'] = False
    
    # 1. 标记valid的subject
    valid_subject = df['股东身份'].isin(['控股股东', '实际控制人'])
    # 注意：由于数据中没有"一致行动人"字段，这里无法直接判断，假设数据中已明确标记
    
    # 2. 标记valid的condition
    # 条件1: 计划通过集中竞价或大宗交易减持
    condition1 = df['减持方式'].isin(['竞价交易', '大宗交易'])
    # 条件2: 首次披露减持计划(存在减持计划且没有历史计划)
    # 需要按股东分组检查是否是首次披露
    df['is_first_plan'] = df.groupby(['公司简称', '股东'])['存在减持计划'].transform(
        lambda x: x & (~x.shift(fill_value=False).cummax())
    )
    condition2 = df['is_first_plan']
    # 条件3: 不存在证监会另有规定的情况(假设数据中没有明确标记，默认为满足)
    condition3 = True
    
    valid_condition = condition1 & condition2 & condition3
    
    # 3. 标记valid的constraint
    # 约束: 最近一期经审计的净利润不为负
    # 假设最近一期经审计的财务报告就是最新的财务数据
    # 需要按公司分组获取最新财务数据
    latest_financials = df.groupby('公司简称').apply(
        lambda x: x.sort_values('日期').iloc[-1][['净利润']]
    ).reset_index()
    latest_financials = latest_financials.rename(columns={'净利润': 'latest_net_profit'})
    
    df = pd.merge(df, latest_financials, on='公司简称', how='left')
    valid_constraint = df['latest_net_profit'] >= 0
    
    # 4. 标记各条件满足情况
    df.loc[valid_subject, 'meu_12_3_subject'] = True
    df.loc[valid_condition, 'meu_12_3_condition'] = True
    df.loc[valid_constraint, 'meu_12_3_constraint'] = True
    
    # 5. 判断违规情况：满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )
    
    # 标记合规性
    df['meu_12_3_compliance'] = True  # 默认合规
    df.loc[is_violation, 'meu_12_3_compliance'] = False
    
    # 清理临时列
    df = df.drop(columns=['is_first_plan', 'latest_net_profit'])
    
    return df

# MEU_4_1


# 身份
你是一个精通法律的程序员. 擅长编写代码以检验特定案例对于特定法律要求的合规性. 

## MEU概念简述
MEU (Minimum Executable Unit) 是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东") 
- condition: 触发条件（如"减持股份")  
- constraint: 约束内容（如"提前15日公告") 
- contextual_info: 补充说明（如价格计算方式）


# 任务
你现在面临一个编写代码以检验某法律事件的合规性的任务, 你需要编写一个函数, 这个函数的输入是一个日频的pandas dataframe, 每一行是在某个交易日内公司的情况和股东的情况, 这个函数会检验该dataframe的每一行在某一个给定的法律最小可执行单元上的合规性, 标注在特定的列上, 并返回标注后的dataframe. 


# 更多要求
 - 你可以进行任意长度的思考, 然后用<CODE></CODE>包裹你的代码. 
 - 你需要按照下面这个框架编写代码: 1. 标记valid的subject(meu_n_k_subject); 2. 标记valid的condition(meu_n_k_condition); 3. 标记valid的constrain(meu_n_k_constrain); 4. 在dataframe中记录subject, condition和constrain的valid情况; 5. 计算整体的违约情况并标记在dataframe上(meu_n_k_compliance). 

# example
下面是一个例子:

## 输入
法律的最小可执行单元编号: MEU_4_1
法律的最小可执行单元内容: {"subject": "上市公司大股东 | 董监高", "condition": "计划通过本所集中竞价或大宗交易减持股份", "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", "contextual_info": NaN}
必要的额外信息: NaN

## 输出
<CODE>
import pandas a pd

def check_meu_4_1(df):
    '''
    代码的注释要清晰, 展示你分别验证subject, condition和constrain的过程.
    "subject": "上市公司大股东 | 董监高", 
    "condition": "计划通过本所集中竞价或大宗交易减持股份", 
    "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", 
    "contextual_info": NaN
    '''

    df['meu_4_2_subject'] = False  # 初始化subject列
    df['meu_4_2_condition'] = False  # 初始化condition列
    df['meu_4_2_constrain'] = False  # 初始化constrain列

    # 标记valid的subject
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'].isin(['董监高'])
    valid_subject = is_major_shareholder | is_director

    # 标记valid的condition
    valid_condition = df['减持方式'].isin(['竞价交易', '大宗交易'])

    # 标记valid的constrain
    df['交易日差'] = (df['计划开始日'] - df['计划披露日']).dt.days
    has_plan = df['存在减持计划']
    valid_constraint = df['交易日差'] >= 15 & df['存在减持计划']

    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_4_1_subject'] = True
    df.loc[valid_condition, 'meu_4_1_condition'] = True
    df.loc[valid_constraint, 'meu_4_1_constraint'] = True

    # 判断违规情况：满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )

    # 标记违规行
    df.loc[is_violation, 'meu_4_1_compliance'] = False

    return df

</CODE>

# 下面轮到你来完成这个任务
法律的最小可执行单元编号: MEU_12_2
法律的最小可执行单元内容: 上市公司控股股东 | 实际控制人 | 一致行动人 | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 | 不得存在下列情形：最近20个交易日内，上市公司任一交易日股票收盘价低于最近一个会计年度或者最近一期财务会计报告期末每股归属于上市公司股东的净资产 | 股票收盘价以最近一个会计年度或者最近一期财务会计报告资产负债表日为基准分别向后复权计算
必要的额外信息: 无


# 附加信息
## 你可以操作的dataframe的columns: 
['日期', '日收益率', '收盘价', '前收盘价', '上市日期', '发行价格', '净资产增长率', '净利润增长率', '净资产',
'净利润', '总股本', '每股净资产', '每股净利润', '公告类型', '公告日期', '复权因子', '收盘价减每股净资产',
'收盘价减每股净利润', '公司涉嫌证券期货违法犯罪事件', '持股比例', '持股数量', '股东身份', '减持方式', '拟减持原因',
'存在减持计划', '计划减持比例', '计划价格下限', '计划价格上限', '当日减持比例', '累计减持比例', '计划披露日',
'计划开始日', '计划结束日', '离任日期', '公司简称', '股东', '股东涉嫌证券期货违法犯罪事件']

## 其中: 
- 所有日期相关的列都已经是pd.datetime格式, ['日期']列本身就是交易日, 不需要额外计算交易日, 但是自然日需要计算. 
- 所有价格的单位都是元而非万元, 千元或百万元; 所有的比例都是小数而非百分数
- 这是一个二维的面板数据, 根据['日期', '公司简称', '股东']可以定位到特定公司的特定股东在特定交易日的状态, 以及公司在该日的状态. 如果你要分公司和股东地考虑一些合规要求, 需要考虑按照 ['公司简称', '股东']分组并在组内完成. 
- 交易方式有['竞价交易', '大宗交易', '融券卖出'], 竞价交易包括集中竞价. 
- ['公司/股东涉嫌证券期货违法犯罪事件'] 有['被中国证监会及其派出机构立案调查', '中国证监会及其派出机构立案调查结束', '行政处罚决定作出', '被司法机关立案侦查', '司法机关立案侦查结束', '被本所公开谴责']
- ['股东身份'] 有 ['董监高', '离任董监高', '控股股东', '实际控制人', '持股5%以上股东', '普通股东'], 这些标签不会并列出现, 因此有时候董监高/离任董监高持股在5%以上但并没有标明'持股5%以上股东', 你需要从['持股比例']手动筛选. 
- ['存在减持计划']是一个布尔值，表示该股东当天是否有减持计划
- 假设公司上市发行价就是最早的非空的收盘价, 公司发行时的控股股东和实际控制人就是第一个非空的收盘价的同一天的控股股东和实际控制人. 
- ['持股比例'] 全部是公开上市前取得的股份. 不考虑承诺和限制出售的股份. 



In [ ]:
import pandas as pd

def check_meu_12_2(df):
    '''
    检查MEU_12_2合规性:
    "subject": "上市公司控股股东 | 实际控制人 | 一致行动人",
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的",
    "constraint": "不得存在下列情形：最近20个交易日内，上市公司任一交易日股票收盘价低于最近一个会计年度或者最近一期财务会计报告期末每股归属于上市公司股东的净资产",
    "contextual_info": "股票收盘价以最近一个会计年度或者最近一期财务会计报告资产负债表日为基准分别向后复权计算"
    '''
    
    # 初始化标记列
    df['meu_12_2_subject'] = False
    df['meu_12_2_condition'] = False
    df['meu_12_2_constraint'] = False
    df['meu_12_2_compliance'] = True  # 默认合规
    
    # 1. 标记valid的subject
    valid_subject = df['股东身份'].isin(['控股股东', '实际控制人']) | \
                   (df['股东'] + df['公司简称']).isin(df.groupby('公司简称')['一致行动人'].first())
    
    # 2. 标记valid的condition
    # 条件1: 计划通过集中竞价或大宗交易减持
    condition1 = df['减持方式'].isin(['竞价交易', '大宗交易'])
    # 条件2: 首次披露减持计划
    condition2 = df['存在减持计划'] & (df.groupby(['公司简称', '股东'])['存在减持计划'].cumcount() == 0)
    # 条件3: 不存在已经按照本指引第四条披露减持计划的情况(假设第四条对应MEU_4_1)
    condition3 = ~df['meu_4_1_compliance'].fillna(True)  # 假设已检查过MEU_4_1
    # 条件4: 中国证监会无其他规定(假设无其他规定)
    condition4 = True
    
    valid_condition = condition1 & condition2 & condition3 & condition4
    
    # 3. 标记valid的constraint
    # 计算复权后的收盘价(假设复权因子已正确计算)
    df['复权收盘价'] = df['收盘价'] * df['复权因子']
    
    # 对每个公司计算最近20个交易日的收盘价是否低于每股净资产
    def check_price_constraint(group):
        # 获取最近一个会计年度或最近一期的每股净资产
        latest_report_date = group['财务会计报告期末'].max()  # 假设有此列
        latest_net_asset_per_share = group.loc[group['财务会计报告期末'] == latest_report_date, '每股净资产'].iloc[0]
        
        # 检查最近20个交易日
        group['price_violation'] = group['复权收盘价'].rolling(20).min() < latest_net_asset_per_share
        return group
    
    # 按公司分组应用检查
    df = df.groupby('公司简称').apply(check_price_constraint)
    
    valid_constraint = ~df['price_violation'].fillna(False)
    
    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_12_2_subject'] = True
    df.loc[valid_condition, 'meu_12_2_condition'] = True
    df.loc[valid_constraint, 'meu_12_2_constraint'] = True
    
    # 判断违规情况：满足主体和条件但不满足约束
    is_violation = valid_subject & valid_condition & (~valid_constraint)
    df.loc[is_violation, 'meu_12_2_compliance'] = False
    
    # 清理临时列
    df.drop(['price_violation', '复权收盘价'], axis=1, inplace=True, errors='ignore')
    
    return df

# MEU_4_1


# 身份
你是一个精通法律的程序员. 擅长编写代码以检验特定案例对于特定法律要求的合规性. 

## MEU概念简述
MEU (Minimum Executable Unit) 是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东") 
- condition: 触发条件（如"减持股份")  
- constraint: 约束内容（如"提前15日公告") 
- contextual_info: 补充说明（如价格计算方式）


# 任务
你现在面临一个编写代码以检验某法律事件的合规性的任务, 你需要编写一个函数, 这个函数的输入是一个日频的pandas dataframe, 每一行是在某个交易日内公司的情况和股东的情况, 这个函数会检验该dataframe的每一行在某一个给定的法律最小可执行单元上的合规性, 标注在特定的列上, 并返回标注后的dataframe. 


# 更多要求
 - 你可以进行任意长度的思考, 然后用<CODE></CODE>包裹你的代码. 
 - 你需要按照下面这个框架编写代码: 1. 标记valid的subject(meu_n_k_subject); 2. 标记valid的condition(meu_n_k_condition); 3. 标记valid的constrain(meu_n_k_constrain); 4. 在dataframe中记录subject, condition和constrain的valid情况; 5. 计算整体的违约情况并标记在dataframe上(meu_n_k_compliance). 

# example
下面是一个例子:

## 输入
法律的最小可执行单元编号: MEU_4_1
法律的最小可执行单元内容: {"subject": "上市公司大股东 | 董监高", "condition": "计划通过本所集中竞价或大宗交易减持股份", "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", "contextual_info": NaN}
必要的额外信息: NaN

## 输出
<CODE>
import pandas a pd

def check_meu_4_1(df):
    '''
    代码的注释要清晰, 展示你分别验证subject, condition和constrain的过程.
    "subject": "上市公司大股东 | 董监高", 
    "condition": "计划通过本所集中竞价或大宗交易减持股份", 
    "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", 
    "contextual_info": NaN
    '''

    df['meu_4_2_subject'] = False  # 初始化subject列
    df['meu_4_2_condition'] = False  # 初始化condition列
    df['meu_4_2_constrain'] = False  # 初始化constrain列

    # 标记valid的subject
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'].isin(['董监高'])
    valid_subject = is_major_shareholder | is_director

    # 标记valid的condition
    valid_condition = df['减持方式'].isin(['竞价交易', '大宗交易'])

    # 标记valid的constrain
    df['交易日差'] = (df['计划开始日'] - df['计划披露日']).dt.days
    has_plan = df['存在减持计划']
    valid_constraint = df['交易日差'] >= 15 & df['存在减持计划']

    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_4_1_subject'] = True
    df.loc[valid_condition, 'meu_4_1_condition'] = True
    df.loc[valid_constraint, 'meu_4_1_constraint'] = True

    # 判断违规情况：满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )

    # 标记违规行
    df.loc[is_violation, 'meu_4_1_compliance'] = False

    return df

</CODE>

# 下面轮到你来完成这个任务
法律的最小可执行单元编号: MEU_12_4
法律的最小可执行单元内容: 控股股东 | 实际控制人（上市后不再具备相关主体身份的） | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 | 不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格 | 股票收盘价以公开发行股票并上市之日为基准向后复权计算
必要的额外信息: 无


# 附加信息
## 你可以操作的dataframe的columns: 
['日期', '日收益率', '收盘价', '前收盘价', '上市日期', '发行价格', '净资产增长率', '净利润增长率', '净资产',
'净利润', '总股本', '每股净资产', '每股净利润', '公告类型', '公告日期', '复权因子', '收盘价减每股净资产',
'收盘价减每股净利润', '公司涉嫌证券期货违法犯罪事件', '持股比例', '持股数量', '股东身份', '减持方式', '拟减持原因',
'存在减持计划', '计划减持比例', '计划价格下限', '计划价格上限', '当日减持比例', '累计减持比例', '计划披露日',
'计划开始日', '计划结束日', '离任日期', '公司简称', '股东', '股东涉嫌证券期货违法犯罪事件']

## 其中: 
- 所有日期相关的列都已经是pd.datetime格式, ['日期']列本身就是交易日, 不需要额外计算交易日, 但是自然日需要计算. 
- 所有价格的单位都是元而非万元, 千元或百万元; 所有的比例都是小数而非百分数
- 这是一个二维的面板数据, 根据['日期', '公司简称', '股东']可以定位到特定公司的特定股东在特定交易日的状态, 以及公司在该日的状态. 如果你要分公司和股东地考虑一些合规要求, 需要考虑按照 ['公司简称', '股东']分组并在组内完成. 
- 交易方式有['竞价交易', '大宗交易', '融券卖出'], 竞价交易包括集中竞价. 
- ['公司/股东涉嫌证券期货违法犯罪事件'] 有['被中国证监会及其派出机构立案调查', '中国证监会及其派出机构立案调查结束', '行政处罚决定作出', '被司法机关立案侦查', '司法机关立案侦查结束', '被本所公开谴责']
- ['股东身份'] 有 ['董监高', '离任董监高', '控股股东', '实际控制人', '持股5%以上股东', '普通股东'], 这些标签不会并列出现, 因此有时候董监高/离任董监高持股在5%以上但并没有标明'持股5%以上股东', 你需要从['持股比例']手动筛选. 
- ['存在减持计划']是一个布尔值，表示该股东当天是否有减持计划
- 假设公司上市发行价就是最早的非空的收盘价, 公司发行时的控股股东和实际控制人就是第一个非空的收盘价的同一天的控股股东和实际控制人. 
- ['持股比例'] 全部是公开上市前取得的股份. 不考虑承诺和限制出售的股份. 



In [ ]:
import pandas as pd

def check_meu_12_4(df):
    '''
    检查MEU_12_4合规性:
    "subject": "控股股东 | 实际控制人（上市后不再具备相关主体身份的）",
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的",
    "constraint": "不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格",
    "contextual_info": "股票收盘价以公开发行股票并上市之日为基准向后复权计算"
    '''
    
    # 初始化标记列
    df['meu_12_4_subject'] = False
    df['meu_12_4_condition'] = False
    df['meu_12_4_constraint'] = True  # 默认满足约束(即不违反)
    df['meu_12_4_compliance'] = True  # 默认合规
    
    # 1. 标记valid的subject
    # 获取每个公司最早的发行价和初始控股股东/实际控制人
    company_info = df.groupby('公司简称').agg({
        '收盘价': 'first',  # 假设发行价就是最早的非空收盘价
        '股东身份': lambda x: x[~x.isna()].iloc[0] if not x.isna().all() else None
    }).rename(columns={'收盘价': '发行价格'})
    
    # 合并发行价信息到原df
    df = df.merge(company_info[['发行价格']], on='公司简称', how='left')
    
    # 判断subject有效性
    is_initial_controller = df['股东身份'].isin(['控股股东', '实际控制人'])
    is_former_controller = ~df['股东身份'].isin(['控股股东', '实际控制人']) & \
                          (df['股东身份'].notna()) & \
                          (df['股东身份'].shift().isin(['控股股东', '实际控制人']))  # 上市后不再具备相关身份
    
    valid_subject = is_initial_controller | is_former_controller
    df.loc[valid_subject, 'meu_12_4_subject'] = True
    
    # 2. 标记valid的condition
    has_plan = df['存在减持计划']
    valid_d_method = df['减持方式'].isin(['竞价交易', '大宗交易'])
    is_first_plan = ~df.duplicated(subset=['公司简称', '股东', '存在减持计划'], keep='first') & has_plan
    
    valid_condition = has_plan & valid_d_method & is_first_plan
    df.loc[valid_condition, 'meu_12_4_condition'] = True
    
    # 3. 标记valid的constraint
    # 计算复权后的收盘价 (假设复权因子已经正确计算)
    df['复权收盘价'] = df['收盘价'] * df['复权因子']
    
    # 对每个公司-股东组合，检查最近20个交易日是否有股价低于发行价的情况
    def check_price_constraint(group):
        # 只检查有减持计划的日子
        plan_days = group[group['存在减持计划']]
        for idx, row in plan_days.iterrows():
            # 获取最近20个交易日
            last_20_days = group[(group['日期'] <= row['日期']) & 
                                (group['日期'] > row['日期'] - pd.Timedelta(days=20))]
            # 检查是否有任何一天复权收盘价低于发行价
            if any(last_20_days['复权收盘价'] < row['发行价格']):
                group.loc[idx, 'meu_12_4_constraint'] = False
        return group
    
    # 按公司和股东分组应用检查
    df = df.groupby(['公司简称', '股东']).apply(check_price_constraint)
    
    # 4. 计算整体合规情况
    # 违规情况：满足主体和条件但不满足约束
    is_violation = (
        df['meu_12_4_subject'] & 
        df['meu_12_4_condition'] & 
        (~df['meu_12_4_constraint'])
    )
    
    df.loc[is_violation, 'meu_12_4_compliance'] = False
    
    return df

# MEU_4_1


# 身份
你是一个精通法律的程序员. 擅长编写代码以检验特定案例对于特定法律要求的合规性. 

## MEU概念简述
MEU (Minimum Executable Unit) 是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东") 
- condition: 触发条件（如"减持股份")  
- constraint: 约束内容（如"提前15日公告") 
- contextual_info: 补充说明（如价格计算方式）


# 任务
你现在面临一个编写代码以检验某法律事件的合规性的任务, 你需要编写一个函数, 这个函数的输入是一个日频的pandas dataframe, 每一行是在某个交易日内公司的情况和股东的情况, 这个函数会检验该dataframe的每一行在某一个给定的法律最小可执行单元上的合规性, 标注在特定的列上, 并返回标注后的dataframe. 


# 更多要求
 - 你可以进行任意长度的思考, 然后用<CODE></CODE>包裹你的代码. 
 - 你需要按照下面这个框架编写代码: 1. 标记valid的subject(meu_n_k_subject); 2. 标记valid的condition(meu_n_k_condition); 3. 标记valid的constrain(meu_n_k_constrain); 4. 在dataframe中记录subject, condition和constrain的valid情况; 5. 计算整体的违约情况并标记在dataframe上(meu_n_k_compliance). 

# example
下面是一个例子:

## 输入
法律的最小可执行单元编号: MEU_4_1
法律的最小可执行单元内容: {"subject": "上市公司大股东 | 董监高", "condition": "计划通过本所集中竞价或大宗交易减持股份", "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", "contextual_info": NaN}
必要的额外信息: NaN

## 输出
<CODE>
import pandas a pd

def check_meu_4_1(df):
    '''
    代码的注释要清晰, 展示你分别验证subject, condition和constrain的过程.
    "subject": "上市公司大股东 | 董监高", 
    "condition": "计划通过本所集中竞价或大宗交易减持股份", 
    "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", 
    "contextual_info": NaN
    '''

    df['meu_4_2_subject'] = False  # 初始化subject列
    df['meu_4_2_condition'] = False  # 初始化condition列
    df['meu_4_2_constrain'] = False  # 初始化constrain列

    # 标记valid的subject
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'].isin(['董监高'])
    valid_subject = is_major_shareholder | is_director

    # 标记valid的condition
    valid_condition = df['减持方式'].isin(['竞价交易', '大宗交易'])

    # 标记valid的constrain
    df['交易日差'] = (df['计划开始日'] - df['计划披露日']).dt.days
    has_plan = df['存在减持计划']
    valid_constraint = df['交易日差'] >= 15 & df['存在减持计划']

    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_4_1_subject'] = True
    df.loc[valid_condition, 'meu_4_1_condition'] = True
    df.loc[valid_constraint, 'meu_4_1_constraint'] = True

    # 判断违规情况：满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )

    # 标记违规行
    df.loc[is_violation, 'meu_4_1_compliance'] = False

    return df

</CODE>

# 下面轮到你来完成这个任务
法律的最小可执行单元编号: MEU_12_1
法律的最小可执行单元内容: 公开发行股票并上市时的控股股东 | 公开发行股票并上市时的实际控制人 | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 | 不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格 | 股票收盘价以公开发行股票并上市之日为基准向后复权计算
必要的额外信息: 无


# 附加信息
## 你可以操作的dataframe的columns: 
['日期', '日收益率', '收盘价', '前收盘价', '上市日期', '发行价格', '净资产增长率', '净利润增长率', '净资产',
'净利润', '总股本', '每股净资产', '每股净利润', '公告类型', '公告日期', '复权因子', '收盘价减每股净资产',
'收盘价减每股净利润', '公司涉嫌证券期货违法犯罪事件', '持股比例', '持股数量', '股东身份', '减持方式', '拟减持原因',
'存在减持计划', '计划减持比例', '计划价格下限', '计划价格上限', '当日减持比例', '累计减持比例', '计划披露日',
'计划开始日', '计划结束日', '离任日期', '公司简称', '股东', '股东涉嫌证券期货违法犯罪事件']

## 其中: 
- 所有日期相关的列都已经是pd.datetime格式, ['日期']列本身就是交易日, 不需要额外计算交易日, 但是自然日需要计算. 
- 所有价格的单位都是元而非万元, 千元或百万元; 所有的比例都是小数而非百分数
- 这是一个二维的面板数据, 根据['日期', '公司简称', '股东']可以定位到特定公司的特定股东在特定交易日的状态, 以及公司在该日的状态. 如果你要分公司和股东地考虑一些合规要求, 需要考虑按照 ['公司简称', '股东']分组并在组内完成. 
- 交易方式有['竞价交易', '大宗交易', '融券卖出'], 竞价交易包括集中竞价. 
- ['公司/股东涉嫌证券期货违法犯罪事件'] 有['被中国证监会及其派出机构立案调查', '中国证监会及其派出机构立案调查结束', '行政处罚决定作出', '被司法机关立案侦查', '司法机关立案侦查结束', '被本所公开谴责']
- ['股东身份'] 有 ['董监高', '离任董监高', '控股股东', '实际控制人', '持股5%以上股东', '普通股东'], 这些标签不会并列出现, 因此有时候董监高/离任董监高持股在5%以上但并没有标明'持股5%以上股东', 你需要从['持股比例']手动筛选. 
- ['存在减持计划']是一个布尔值，表示该股东当天是否有减持计划
- 假设公司上市发行价就是最早的非空的收盘价, 公司发行时的控股股东和实际控制人就是第一个非空的收盘价的同一天的控股股东和实际控制人. 
- ['持股比例'] 全部是公开上市前取得的股份. 不考虑承诺和限制出售的股份. 



In [ ]:
import pandas as pd

def check_meu_12_1(df):
    '''
    检查MEU_12_1合规性:
    "subject": "公开发行股票并上市时的控股股东 | 公开发行股票并上市时的实际控制人",
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的",
    "constraint": "不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格",
    "contextual_info": "股票收盘价以公开发行股票并上市之日为基准向后复权计算"
    '''
    
    # 初始化标记列
    df['meu_12_1_subject'] = False
    df['meu_12_1_condition'] = False
    df['meu_12_1_constraint'] = False
    df['meu_12_1_compliance'] = True  # 默认合规
    
    # 1. 标记valid的subject
    # 获取每个公司上市时的控股股东和实际控制人
    def get_ipo_holders(group):
        # 找到公司上市日期(第一个非空收盘价的日期)
        ipo_date = group['上市日期'].min()
        ipo_data = group[group['日期'] == ipo_date].iloc[0]
        return {
            '控股股东': ipo_data['控股股东'] if pd.notna(ipo_data['控股股东']) else None,
            '实际控制人': ipo_data['实际控制人'] if pd.notna(ipo_data['实际控制人']) else None,
            '发行价格': ipo_data['发行价格']
        }
    
    ipo_info = df.groupby('公司简称').apply(get_ipo_holders).to_dict()
    
    # 标记subject
    for idx, row in df.iterrows():
        company = row['公司简称']
        holder = row['股东']
        if company in ipo_info:
            if (holder == ipo_info[company]['控股股东'] or 
                holder == ipo_info[company]['实际控制人']):
                df.at[idx, 'meu_12_1_subject'] = True
    
    # 2. 标记valid的condition
    # 条件: 有减持计划且减持方式为竞价交易或大宗交易
    valid_condition = (
        df['存在减持计划'] & 
        df['减持方式'].isin(['竞价交易', '大宗交易'])
    )
    
    # 首次披露减持计划的条件(在组内按日期排序)
    def mark_first_disclosure(group):
        group = group.sort_values('日期')
        first_disclosure_idx = group[group['存在减持计划']].index.min()
        if pd.notna(first_disclosure_idx):
            group.loc[first_disclosure_idx:, 'meu_12_1_condition'] = True
        return group
    
    df = df.groupby(['公司简称', '股东'], group_keys=False).apply(mark_first_disclosure)
    df['meu_12_1_condition'] = df['meu_12_1_condition'] & valid_condition
    
    # 3. 标记valid的constraint
    # 约束: 最近20个交易日内收盘价不低于发行价
    def check_price_constraint(group):
        company = group['公司简称'].iloc[0]
        if company not in ipo_info:
            return group
        
        ipo_price = ipo_info[company]['发行价格']
        
        # 计算每个交易日最近20个交易日的收盘价是否都>=发行价
        for idx, row in group.iterrows():
            current_date = row['日期']
            # 获取最近20个交易日(包括当前日)
            last_20 = group[
                (group['日期'] <= current_date) & 
                (group['日期'] > current_date - pd.Timedelta(days=20))
            ]
            if len(last_20) > 0:
                # 检查是否有任何一天收盘价低于发行价
                has_violation = any(last_20['收盘价'] < ipo_price)
                group.at[idx, 'meu_12_1_constraint'] = not has_violation
        
        return group
    
    df = df.groupby(['公司简称', '股东'], group_keys=False).apply(check_price_constraint)
    
    # 4. 计算整体合规情况
    # 违规情况: 满足主体和条件但不满足约束
    is_violation = (
        df['meu_12_1_subject'] & 
        df['meu_12_1_condition'] & 
        (~df['meu_12_1_constraint'])
    )
    df.loc[is_violation, 'meu_12_1_compliance'] = False
    
    return df

# MEU_4_1


# 身份
你是一个精通法律的程序员. 擅长编写代码以检验特定案例对于特定法律要求的合规性. 

## MEU概念简述
MEU (Minimum Executable Unit) 是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东") 
- condition: 触发条件（如"减持股份")  
- constraint: 约束内容（如"提前15日公告") 
- contextual_info: 补充说明（如价格计算方式）


# 任务
你现在面临一个编写代码以检验某法律事件的合规性的任务, 你需要编写一个函数, 这个函数的输入是一个日频的pandas dataframe, 每一行是在某个交易日内公司的情况和股东的情况, 这个函数会检验该dataframe的每一行在某一个给定的法律最小可执行单元上的合规性, 标注在特定的列上, 并返回标注后的dataframe. 


# 更多要求
 - 你可以进行任意长度的思考, 然后用<CODE></CODE>包裹你的代码. 
 - 你需要按照下面这个框架编写代码: 1. 标记valid的subject(meu_n_k_subject); 2. 标记valid的condition(meu_n_k_condition); 3. 标记valid的constrain(meu_n_k_constrain); 4. 在dataframe中记录subject, condition和constrain的valid情况; 5. 计算整体的违约情况并标记在dataframe上(meu_n_k_compliance). 

# example
下面是一个例子:

## 输入
法律的最小可执行单元编号: MEU_4_1
法律的最小可执行单元内容: {"subject": "上市公司大股东 | 董监高", "condition": "计划通过本所集中竞价或大宗交易减持股份", "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", "contextual_info": NaN}
必要的额外信息: NaN

## 输出
<CODE>
import pandas a pd

def check_meu_4_1(df):
    '''
    代码的注释要清晰, 展示你分别验证subject, condition和constrain的过程.
    "subject": "上市公司大股东 | 董监高", 
    "condition": "计划通过本所集中竞价或大宗交易减持股份", 
    "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", 
    "contextual_info": NaN
    '''

    df['meu_4_2_subject'] = False  # 初始化subject列
    df['meu_4_2_condition'] = False  # 初始化condition列
    df['meu_4_2_constrain'] = False  # 初始化constrain列

    # 标记valid的subject
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'].isin(['董监高'])
    valid_subject = is_major_shareholder | is_director

    # 标记valid的condition
    valid_condition = df['减持方式'].isin(['竞价交易', '大宗交易'])

    # 标记valid的constrain
    df['交易日差'] = (df['计划开始日'] - df['计划披露日']).dt.days
    has_plan = df['存在减持计划']
    valid_constraint = df['交易日差'] >= 15 & df['存在减持计划']

    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_4_1_subject'] = True
    df.loc[valid_condition, 'meu_4_1_condition'] = True
    df.loc[valid_constraint, 'meu_4_1_constraint'] = True

    # 判断违规情况：满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )

    # 标记违规行
    df.loc[is_violation, 'meu_4_1_compliance'] = False

    return df

</CODE>

# 下面轮到你来完成这个任务
法律的最小可执行单元编号: MEU_12_5
法律的最小可执行单元内容: 公开发行时持股5%以上的第一大股东 | 一致行动人 | 上市公司在公开发行股票并上市时披露为无控股股东、实际控制人, 且计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 | 不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格 | 股票收盘价以公开发行股票并上市之日为基准向后复权计算
必要的额外信息: 无


# 附加信息
## 你可以操作的dataframe的columns: 
['日期', '日收益率', '收盘价', '前收盘价', '上市日期', '发行价格', '净资产增长率', '净利润增长率', '净资产',
'净利润', '总股本', '每股净资产', '每股净利润', '公告类型', '公告日期', '复权因子', '收盘价减每股净资产',
'收盘价减每股净利润', '公司涉嫌证券期货违法犯罪事件', '持股比例', '持股数量', '股东身份', '减持方式', '拟减持原因',
'存在减持计划', '计划减持比例', '计划价格下限', '计划价格上限', '当日减持比例', '累计减持比例', '计划披露日',
'计划开始日', '计划结束日', '离任日期', '公司简称', '股东', '股东涉嫌证券期货违法犯罪事件']

## 其中: 
- 所有日期相关的列都已经是pd.datetime格式, ['日期']列本身就是交易日, 不需要额外计算交易日, 但是自然日需要计算. 
- 所有价格的单位都是元而非万元, 千元或百万元; 所有的比例都是小数而非百分数
- 这是一个二维的面板数据, 根据['日期', '公司简称', '股东']可以定位到特定公司的特定股东在特定交易日的状态, 以及公司在该日的状态. 如果你要分公司和股东地考虑一些合规要求, 需要考虑按照 ['公司简称', '股东']分组并在组内完成. 
- 交易方式有['竞价交易', '大宗交易', '融券卖出'], 竞价交易包括集中竞价. 
- ['公司/股东涉嫌证券期货违法犯罪事件'] 有['被中国证监会及其派出机构立案调查', '中国证监会及其派出机构立案调查结束', '行政处罚决定作出', '被司法机关立案侦查', '司法机关立案侦查结束', '被本所公开谴责']
- ['股东身份'] 有 ['董监高', '离任董监高', '控股股东', '实际控制人', '持股5%以上股东', '普通股东'], 这些标签不会并列出现, 因此有时候董监高/离任董监高持股在5%以上但并没有标明'持股5%以上股东', 你需要从['持股比例']手动筛选. 
- ['存在减持计划']是一个布尔值，表示该股东当天是否有减持计划
- 假设公司上市发行价就是最早的非空的收盘价, 公司发行时的控股股东和实际控制人就是第一个非空的收盘价的同一天的控股股东和实际控制人. 
- ['持股比例'] 全部是公开上市前取得的股份. 不考虑承诺和限制出售的股份. 



In [ ]:
import pandas as pd

def check_meu_12_5(df):
    '''
    检查MEU_12_5合规性:
    "subject": "公开发行时持股5%以上的第一大股东 | 一致行动人 | 上市公司在公开发行股票并上市时披露为无控股股东、实际控制人",
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的",
    "constraint": "不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格",
    "contextual_info": "股票收盘价以公开发行股票并上市之日为基准向后复权计算"
    '''
    
    # 初始化标记列
    df['meu_12_5_subject'] = False
    df['meu_12_5_condition'] = False
    df['meu_12_5_constraint'] = True  # 默认满足约束
    df['meu_12_5_compliance'] = True  # 默认合规
    
    # 1. 标记valid的subject
    # 获取每个公司的上市日期和发行价格
    company_info = df.groupby('公司简称').agg({
        '上市日期': 'first',
        '发行价格': 'first',
        '股东身份': lambda x: x[df['上市日期'] == df['上市日期'].min()].unique()
    }).reset_index()
    
    # 标记公开发行时持股5%以上的第一大股东
    for company in df['公司简称'].unique():
        # 获取该公司上市日期
        ipo_date = company_info.loc[company_info['公司简称'] == company, '上市日期'].iloc[0]
        
        # 获取上市当天的股东数据
        ipo_day_df = df[(df['公司简称'] == company) & (df['日期'] == ipo_date)]
        
        # 找出持股5%以上的股东
        major_shareholders = ipo_day_df[ipo_day_df['持股比例'] >= 0.05]
        
        # 如果没有持股5%以上的股东，则检查是否披露为无控股股东、实际控制人
        if len(major_shareholders) == 0:
            no_control = True
        else:
            # 找出第一大股东
            top_shareholder = major_shareholders.loc[major_shareholders['持股比例'].idxmax()]
            no_control = False
        
        # 标记subject
        for idx, row in df[df['公司简称'] == company].iterrows():
            # 如果是第一大股东
            if len(major_shareholders) > 0 and row['股东'] == top_shareholder['股东']:
                df.loc[idx, 'meu_12_5_subject'] = True
            # 如果是无控股股东的情况
            elif no_control:
                df.loc[idx, 'meu_12_5_subject'] = True
    
    # 2. 标记valid的condition
    # 条件1: 计划通过集中竞价交易或大宗交易减持股份
    valid_trade_type = df['减持方式'].isin(['竞价交易', '大宗交易'])
    
    # 条件2: 首次披露减持计划
    # 需要按股东分组，找出每个股东的第一个减持计划
    first_plan_dates = df[df['存在减持计划']].groupby(['公司简称', '股东'])['计划披露日'].min()
    
    for idx, row in df.iterrows():
        company = row['公司简称']
        shareholder = row['股东']
        plan_date = row['计划披露日']
        
        # 检查是否是首次披露
        if (company, shareholder) in first_plan_dates and plan_date == first_plan_dates[(company, shareholder)]:
            is_first_plan = True
        else:
            is_first_plan = False
            
        # 综合条件
        df.loc[idx, 'meu_12_5_condition'] = (valid_trade_type[idx] and 
                                           is_first_plan and 
                                           row['存在减持计划'])
    
    # 3. 标记valid的constraint
    # 检查最近20个交易日是否有收盘价低于发行价
    for company in df['公司简称'].unique():
        # 获取该公司发行价
        ipo_price = company_info.loc[company_info['公司简称'] == company, '发行价格'].iloc[0]
        
        # 获取该公司所有交易日数据
        company_df = df[df['公司简称'] == company].sort_values('日期')
        
        # 计算复权收盘价 (假设复权因子已经正确计算)
        company_df['复权收盘价'] = company_df['收盘价'] * company_df['复权因子']
        
        # 对每个交易日，检查前20个交易日的复权收盘价是否低于发行价
        for idx, row in company_df.iterrows():
            current_date = row['日期']
            
            # 获取前20个交易日
            last_20_days = company_df[
                (company_df['日期'] <= current_date) & 
                (company_df['日期'] > current_date - pd.Timedelta(days=20))
            ]
            
            # 检查是否有任何一天的复权收盘价低于发行价
            if any(last_20_days['复权收盘价'] < ipo_price):
                df.loc[idx, 'meu_12_5_constraint'] = False
    
    # 4. 计算整体合规情况
    # 违规情况：满足主体和条件但不满足约束
    is_violation = (
        df['meu_12_5_subject'] & 
        df['meu_12_5_condition'] & 
        (~df['meu_12_5_constraint'])
    )
    
    df.loc[is_violation, 'meu_12_5_compliance'] = False
    
    return df